In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files

# **Process Multigrams for Training Word-Embedding Models**

## **Goal**: Download and preprocess mulitgrams for use in training `Word2Vec` models. 

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace. I still run up against time and time and resource limits and have designed the code to contend with them.

The code affords some options to keep things efficient. Throughout the workflow you can specify compress=True, which tells the code to compress its output files. Downstream scripts will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once that step is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

**NOTE:** You'll probably want to have run the unigram workflow before processing multigrams. That workflow allows you create a vocabulary file for filtering out uncommon tokens from the multigrams. Although you can run the `filter_ngrams` module without a vocab file, most use cases will benefit from one.

### Download multigrams
Here, I'm using `download_ngrams` module to fetch 5grams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). I've specified `compress=True` becausae 5gram files are _big_.

`[Runtime: 4:19:11.661609]`

In [3]:
download_ngram_files(
    ngram_size=5,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False
)

Start Time:                2025-01-04 17:43:25.557908

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng/eng-5-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1download
File index range:          0 to 19422
File URLs available:       19423
File URLs to use:          19423
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng/5-00000-of-19423.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng/5-19422-of-19423.gz
Ngram size:                5
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  False


End Time:                  2025-01-04 22:02:37.219517
Total runtime:             4:19:11.661609



### Convert files from TXT to JSONL
This module converts the original multigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

`[Total runtime: 1:30:56.083010]`

In [3]:
convert_to_jsonl_files(
    ngram_size=5,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-05 13:47:05.912405

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/2convert
File index range:          0 to 19422
Files available:           19423
Files to use:              19423
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1download/5-00000-of-19423.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1download/5-19422-of-19423.txt.lz4
Ngram size:                5
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False
Delete input directory:    True

Converting      |██████████████████████████████████████████████████| 100.0% 19423       /19423      

End Time:                  2025-01-05 15:19:30.905972
Total runtime:             1:

### Make multigrams all lowercase
This module lowercases all characters in the multigrams. Most use cases benefit from this.

`[Total runtime: 0:47:33.859616]`

In [4]:
lowercase_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-05 15:19:31.052375

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/3lowercase
File index range:          0 to 6520
Files available:           6521
Files to use:              6521
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/2convert/5-00000-of-19423.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/2convert/5-19422-of-19423.jsonl.lz4
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False
Delete input directory:    True

Lowercasing     |██████████████████████████████████████████████████| 100.0% 6521        /6521       

End Time:                  2025-01-05 16:07:13.575473
Total runtime:             0:47:42.523098



### Lemmatize the multigrams
Likewise, most use cases will benefit from multigrams that are lemmatized—that is, reduced to their base form. This requires POS-tagged multigrams. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. The POS tag will then be discarded as it is no longer useful.

`[Total runtime: 1:39:30.015704]`

In [2]:
lemmatize_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-05 16:40:58.283173

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/4lemmatize
File index range:          0 to 6520
Files available:           6521
Files to use:              6521
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/3lowercase/5-00000-of-19423.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/3lowercase/5-19422-of-19423.jsonl.lz4
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False
Delete input directory:    True

Lemmatizing     |██████████████████████████████████████████████████| 100.0% 6521        /6521       

End Time:                  2025-01-05 18:18:46.729100
Total runtime:             1:37:48.445927



### Filter the multigrams
This module removes tokens that provide little information about words' semantic context—specifically, those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`). You can also specify a **vocabulary file** like the one illustrated in the unigram workflow. A vocabulary file is simply a list of the _N_ most common words in the unigram corpus; the multigram tokens are checked against this list and those that don't appear in it are dropped.

The filtering process will inevitably turn some longer ngrams (e.g., 5grams) into shorter ones (e.g., 3grams) after unwanted tokens are dropped. The training of word-embedding models requires _linguistic context_—which in turn requires ngrams containing more than one token. A unigram isn't useful for helping a model learn what "company" a word keeps. Thus, the `min_tokens` option allows you to drop ngrams that fall below a specified length during filtering. If filtering results in an ngram with fewer than the minimum tokens, all data for that ngram is dropped entirely. Here, I've set `min_tokens=2`, since two tokens (and higher) provide at least some contextual information.

`[Total runtime: 0:49:53.902454]`

In [3]:
filter_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=2,
    vocab_file='/vast/edk202/NLP_corpora/Googl_Books/20200217/eng/',
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                   2025-01-05 18:18:46.742270

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5filter
File index range:             0 to 6520
Files available:              6521
Files to use:                 6521
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/4lemmatize/5-00000-of-19423.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/4lemmatize/5-19422-of-19423.jsonl.lz4
Ngram size:                   5
Number of workers:            48
Compress output files:        True
Overwrite existing files:     False
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:          True
Drop ngrams under

### Sort and combine the multigram files
This modules creates a single, fully-sorted multigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

Because this step can take a _very_ long time for larger multigrams (e.g., 5grams), we can run it in sessions using the `start_iteration` and `end_iteration` options. Iteration 1 comes after the initial file sort. If you only have time to complete, say, iterations 1–3, you can set `end_iteration=3`. During a later session, you can specify `start_iteration=4` to pick up where you left off.

`[Sort + Iteration 1 runtime: 1:44:39.319527]`

`[Iteration 2 runtime:        0:47:37.115564]`

`[Iteration 3 runtime:        1:01:02.813422]`

`[Iteration 4 runtime:        1:14:05.632074]`

`[Iteration 5 runtime:        2:21:00.000000]`

`[Iteration 6 runtime:        4:26:40.465811]`

`[Iteration 7 runtime:        7:00:51.409004]`

In [2]:
sort_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=10,
    sort_key='ngram',
    compress=True,
    overwrite=False,
    sort_order='ascending',
    start_iteration=7,
    end_iteration=7,
    delete_input=True
)

Start Time:                2025-01-07 14:48:30.163232

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/6corpus/5gram-merged.jsonl.lz4
Files available:           6520
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5filter/5-00069-of-19423.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5filter/5-19422-of-19423.jsonl.lz4
Files to use:              6520
Ngram size:                5
Number of workers:         10
Compress output files:     True
Overwrite existing files:  False
Sort key:                  ngram
Sort order:                ascending
Heap-merge start it

### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [ ]:
check_file_sorted(
    input_file='/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/6corpus/5gram-merged.jsonl.lz4',
    field="ngram",
    sort_order="ascending"
)

### Consolidate duplicate multigrams
This module consolidates the sorted multigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime: 0:12:33.459340]`

In [2]:
consolidate_duplicate_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=14,
    lines_per_chunk=1000000,
    compress=True,
    overwrite=False
)

Creating and Processing Chunks: 1982 chunks [4:25:35,  8.04s/ chunks]


Merging chunks...
Merging completed.

End Time: 2025-01-09 05:09:36.088154
Total runtime: 4:58:07.036475


### View line [OPTIONAL]
If we want, we can inspect a line in the file.

In [5]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '5gram_files/6corpus/5gram-corpus.jsonl.lz4'
    ),
    start_line=1100050,
    end_line=1100060,
    parse_json=True
)

Line 1100050: {'ngram': 'lumbar puncture yield turbid fluid', 'freq_tot': 45, 'doc_tot': 44, 'freq': {'1906': 1, '1908': 7, '1909': 2, '1912': 4, '1914': 1, '1915': 5, '1916': 9, '1928': 3, '1931': 2, '1932': 1, '1933': 1, '1938': 2, '1941': 2, '1944': 1, '1946': 1, '1947': 1, '1964': 1, '1965': 1}, 'doc': {'1906': 1, '1908': 7, '1909': 2, '1912': 4, '1914': 1, '1915': 5, '1916': 9, '1928': 3, '1931': 2, '1932': 1, '1933': 1, '1938': 1, '1941': 2, '1944': 1, '1946': 1, '1947': 1, '1964': 1, '1965': 1}}
Line 1100051: {'ngram': 'lumbar radicular pain', 'freq_tot': 910, 'doc_tot': 672, 'freq': {'1983': 60, '1986': 7, '1987': 12, '1991': 11, '1992': 25, '1993': 17, '1994': 7, '1995': 21, '1996': 20, '1998': 9, '2002': 24, '2003': 17, '2005': 36, '2006': 25, '2007': 46, '2008': 68, '2010': 45, '2011': 72, '2012': 53, '2013': 49, '2014': 27, '2015': 44, '2016': 18, '2017': 23, '2018': 50, '2019': 29, '1972': 2, '1989': 17, '1990': 9, '1999': 7, '2004': 22, '2009': 14, '1985': 3, '1988': 4, '